# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [6]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.2727490810240811
 2.3783368266387597
 2.2838288271103773

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [7]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.567866  1.0       0.0
 0.977184  0.582597  1.0
U factor:
3×3 Matrix{Float64}:
 0.680846  0.28801   0.303893
 0.0       0.803899  0.757179
 0.0       0.0       0.225147

In [8]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [9]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [10]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.567866  1.0       0.0
 0.977184  0.582597  1.0

In [11]:
Alu.U

3×3 Matrix{Float64}:
 0.680846  0.28801   0.303893
 0.0       0.803899  0.757179
 0.0       0.0       0.225147

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [12]:
A\b

3-element Vector{Float64}:
 1.0
 0.9999999999999987
 1.000000000000001

In [13]:
Alu\b

3-element Vector{Float64}:
 1.0
 0.9999999999999987
 1.000000000000001

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [14]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [15]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 -1.02746  -1.04041   -1.17496
  0.0      -0.706145  -0.689368
  0.0       0.0       -0.169847

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [16]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

In [17]:
Aqr.R

3×3 Matrix{Float64}:
 -1.02746  -1.04041   -1.17496
  0.0      -0.706145  -0.689368
  0.0       0.0       -0.169847

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [18]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.3028278166356104
  0.9185143768757014
  4.10508098016962
vectors:
3×3 Matrix{Float64}:
  0.177614   0.904038  -0.388803
 -0.761571  -0.123954  -0.636117
  0.623268  -0.409085  -0.666473

The values and the vectors can be extracted from the Eigen type by special indexing

In [19]:
AsymEig.values

3-element Vector{Float64}:
 -0.3028278166356104
  0.9185143768757014
  4.10508098016962

In [20]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.177614   0.904038  -0.388803
 -0.761571  -0.123954  -0.636117
  0.623268  -0.409085  -0.666473

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [21]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          -2.88658e-15   5.10703e-15
 4.44089e-16   1.0          -1.28786e-14
 3.33067e-16  -5.10703e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [22]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [73]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [80]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-1.839680726967838

In [81]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [82]:
Asym_explicit = Symmetric(Asym_noisy);
println(Asym_explicit[1,2] == Asym_explicit[2,1])
println(Asym_noisy[2,1] == Asym_noisy[1,2])

true
false


Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [83]:
@time eigvals(Asym);

  0.062228 seconds (11 allocations: 7.989 MiB)


In [84]:
@time eigvals(Asym_noisy);

  0.388800 seconds (14 allocations: 7.928 MiB)


In [85]:
@time eigvals(Asym_explicit);

  0.061912 seconds (11 allocations: 7.989 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [86]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.657423 seconds (346.46 k allocations: 206.999 MiB, 4.80% gc time, 28.81% compilation time)


6.719672531631735

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [87]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [91]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 4//5   4//5  2//5
 3//10  4//5  3//10
 3//5   1//2  7//10

In [92]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  2
 7//5
 9//5

In [93]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1
 1
 1

In [94]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
  1      0    0
 3//8    1    0
 3//4  -1//5  1
U factor:
3×3 Matrix{Rational{BigInt}}:
 4//5  4//5   2//5
  0    1//2   3//20
  0     0    43//100

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [95]:
using LinearAlgebra

In [163]:
A = [ 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36]

A_eigv = eigvals(A)
println(size(A_eigv))
print(typeof([-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]))
round.(A_eigv) - round.([-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143])

(5,)
Vector{Float64}

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [162]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [161]:
A_diag = diagm(A_eigv)

A_diag - [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

5×5 Matrix{Float64}:
 -0.000227648  0.0          0.0         0.0          0.0
  0.0          1.54469e-5   0.0         0.0          0.0
  0.0          0.0         -3.27207e-5  0.0          0.0
  0.0          0.0          0.0         1.47751e-5   0.0
  0.0          0.0          0.0         0.0         -0.000269853

In [160]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

LoadError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [158]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [159]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]